In [1]:
import open3d as o3d
import numpy as np

file = '../../../UrbanNav/HK-Data20200314/velodyne/549935.7771.ply'

pcd = o3d.io.read_point_cloud(file)
# o3d.visualization.draw_geometries([pcd])

In [9]:
n_scan = 32
horizon_scan = 1800
ang_top = 30
ang_bottom = -10
ang_res_x = 360/horizon_scan
ang_res_y = (ang_top-ang_bottom)/n_scan
ground_scan_ind = 20
sensor_mount_angle = 0
sensor_minimum_range = 1

pcd = o3d.io.read_point_cloud(file)
coords = np.asarray(pcd.points)

# Reorder points
reordered_coords = np.zeros(shape=[horizon_scan * n_scan, 4])
for [x, y, z] in coords:
    if np.linalg.norm([x, y, z]) < sensor_minimum_range:
        continue

    vertical_angle = np.arctan2(z, np.linalg.norm([x, y])) * 180.0 / np.pi

    if vertical_angle < ang_bottom or vertical_angle > ang_top:
        continue

    vert_id = np.round((vertical_angle - ang_bottom) / ang_res_y)

    horizon_angle = np.arctan2(y, x) * 180.0 / np.pi

    if horizon_angle < 0:
        horizon_angle += 360

    horizon_id = np.round(horizon_angle / ang_res_x)

    index = int(horizon_id + vert_id * horizon_scan)

    reordered_coords[index, :3] = [x, y, z]

# Ground removal
for i in range(reordered_coords.shape[0]):
    lower_id = i
    upper_id = i + horizon_scan

    if upper_id >= reordered_coords.shape[0]:
        continue

    diffX = reordered_coords[upper_id, 0] - reordered_coords[lower_id, 0]
    diffY = reordered_coords[upper_id, 1] - reordered_coords[lower_id, 1]
    diffZ = reordered_coords[upper_id, 2] - reordered_coords[lower_id, 2]

    angle = np.arctan2(diffZ, np.linalg.norm([diffX, diffY])) * 180.0 / np.pi

    if np.abs(angle - sensor_minimum_range) <= 10:
        reordered_coords[i, 3] = 1

reordered_coords = reordered_coords[np.logical_not(np.logical_and(reordered_coords[:, 0] == 0, reordered_coords[:, 1] == 0, reordered_coords[:, 2] == 0))]
segmented_coords = reordered_coords[:, :3][reordered_coords[:, -1] != 1]


[[ 3.185295  0.003892 -0.523325  0.      ]
 [ 3.189218  0.012803 -0.523974  0.      ]
 [ 3.189162  0.022821 -0.523974  0.      ]
 ...
 [ 3.164147 -0.033688  0.59619   0.      ]
 [ 3.170129 -0.024345  0.597301  0.      ]
 [ 3.168219 -0.015483  0.59693   0.      ]]


In [17]:
pcd_seg = o3d.geometry.PointCloud()
pcd_seg.points = o3d.utility.Vector3dVector(segmented_coords)
o3d.visualization.draw_geometries([pcd_seg])